In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from plotly import graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
# Le fichier comporte une ligne d'en-tête supplémentaire qui réduit l'ouverture du fichier à une seule colonne après export
# Or nous voulons avoir accès à l'intégralité des données pour une première exploration
# Nous allons utiliser l'argument skiprows pour ne pas prendre en compte cet en-tête
df = pd.read_csv('SH.Ts+dSST.csv', sep = ',', skiprows=1)
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,0.00,0.04,.06,-.01,-.12,-.24,-.16,.07,-.04,-.14,.00,.06,-.04,***,***,-.02,-.11,-.06
1,1881,-0.08,-0.06,.10,.10,.09,-.05,-.07,-.02,-.03,.00,.00,.10,.01,.00,-.03,.10,-.05,-.01
2,1882,0.07,0.08,.07,-.01,-.04,-.15,-.04,.01,-.04,.04,-.01,-.07,-.01,.01,.08,.01,-.06,.00
3,1883,-0.02,-0.08,-.08,-.07,-.10,-.02,-.08,-.05,-.10,-.05,-.04,-.05,-.06,-.06,-.05,-.08,-.05,-.06
4,1884,-0.08,-0.05,-.09,-.21,-.31,-.29,-.21,-.06,-.10,-.06,-.10,-.15,-.14,-.13,-.06,-.20,-.19,-.09


In [3]:
# On examine les informations disponibles pour observer les types de données et éventuelles valeurs manquantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    145 non-null    int64  
 1   Jan     145 non-null    float64
 2   Feb     145 non-null    float64
 3   Mar     145 non-null    object 
 4   Apr     145 non-null    object 
 5   May     145 non-null    object 
 6   Jun     145 non-null    object 
 7   Jul     145 non-null    object 
 8   Aug     145 non-null    object 
 9   Sep     145 non-null    object 
 10  Oct     145 non-null    object 
 11  Nov     145 non-null    object 
 12  Dec     145 non-null    object 
 13  J-D     145 non-null    object 
 14  D-N     145 non-null    object 
 15  DJF     145 non-null    object 
 16  MAM     145 non-null    object 
 17  JJA     145 non-null    object 
 18  SON     145 non-null    object 
dtypes: float64(2), int64(1), object(16)
memory usage: 21.7+ KB


In [4]:
# On remarque dès les premières lignes du tableau que des valeurs sont manquantes mais pas sous codification N/A ou NaN
# Nous allons opérer une recherche sur toutes les colonnes de type object pour vérifier les occurences de '***'
# L'idée étant de récupérer le pourcentage de données impactées afin de décider quoi faire de ces singularités.

variables = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec','J-D', 'D-N', 'DJF', 'MAM', 'JJA', 'SON']
missing_data = df[variables].eq('***')
print(missing_data.mean()*100)

Jan    0.000000
Feb    0.000000
Mar    0.689655
Apr    0.689655
May    0.689655
Jun    0.689655
Jul    0.689655
Aug    0.689655
Sep    0.689655
Oct    0.689655
Nov    0.689655
Dec    0.689655
J-D    0.689655
D-N    1.379310
DJF    0.689655
MAM    0.689655
JJA    0.689655
SON    0.689655
dtype: float64


In [5]:
# L'ensemble de ces colonnes semblent impactées. Ce sont soit des colonnes présentant une température relevée mensuelle,
# soit des colonnes qui condensent en moyenne les résultats des mois de l'année étudiée. 
# Il existe très peu de données manquantes mais la manière choisie
# pour les remplacer pourrait avoir un impact sur les futurs calculs et rendus.Nous y reviendrons.

# Procédons d'abord au conversion aux bons formats pour chaque variable.

In [6]:
# On remarque que les deux premiers mois (Jan, Feb) sont au format float64 quand les 10 autres sont au format object
# Cela va nous poser des problèmes lors du traitement et l'analyse des données. Par ailleurs,
# ce format nous empêche de vérifier globalement si des valeurs sont effectivement manquantes.
# Il s'agit donc d'opérer une conversion de tous les objects mois vers un format float

months = ['Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
df[months] =df[months].apply(pd.to_numeric, errors='coerce')

# on revérifie que la conversion s'est bien opérée :
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    145 non-null    int64  
 1   Jan     145 non-null    float64
 2   Feb     145 non-null    float64
 3   Mar     144 non-null    float64
 4   Apr     144 non-null    float64
 5   May     144 non-null    float64
 6   Jun     144 non-null    float64
 7   Jul     144 non-null    float64
 8   Aug     144 non-null    float64
 9   Sep     144 non-null    float64
 10  Oct     144 non-null    float64
 11  Nov     144 non-null    float64
 12  Dec     144 non-null    float64
 13  J-D     145 non-null    object 
 14  D-N     145 non-null    object 
 15  DJF     145 non-null    object 
 16  MAM     145 non-null    object 
 17  JJA     145 non-null    object 
 18  SON     145 non-null    object 
dtypes: float64(12), int64(1), object(6)
memory usage: 21.7+ KB


In [7]:
# Il en est de même pour les dernières colonnes du tableau, qui sont des agrégats et devraient être de type numériques pour 
# maintenir la cohérence avec les colonnes présentant les températures mensuelles.

other_variables = ['J-D', 'D-N', 'DJF', 'MAM', 'JJA', 'SON']
df[other_variables] =df[other_variables].apply(pd.to_numeric, errors='coerce')

In [8]:
# on revérifie que la conversion s'est bien opérée :
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    145 non-null    int64  
 1   Jan     145 non-null    float64
 2   Feb     145 non-null    float64
 3   Mar     144 non-null    float64
 4   Apr     144 non-null    float64
 5   May     144 non-null    float64
 6   Jun     144 non-null    float64
 7   Jul     144 non-null    float64
 8   Aug     144 non-null    float64
 9   Sep     144 non-null    float64
 10  Oct     144 non-null    float64
 11  Nov     144 non-null    float64
 12  Dec     144 non-null    float64
 13  J-D     144 non-null    float64
 14  D-N     143 non-null    float64
 15  DJF     144 non-null    float64
 16  MAM     144 non-null    float64
 17  JJA     144 non-null    float64
 18  SON     144 non-null    float64
dtypes: float64(18), int64(1)
memory usage: 21.7 KB


In [9]:
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,0.00,0.04,0.06,-0.01,-0.12,-0.24,-0.16,0.07,-0.04,-0.14,0.00,0.06,-0.04,NaN,NaN,-0.02,-0.11,-0.06
1,1881,-0.08,-0.06,0.10,0.10,0.09,-0.05,-0.07,-0.02,-0.03,0.00,0.00,0.10,0.01,0.00,-0.03,0.10,-0.05,-0.01
2,1882,0.07,0.08,0.07,-0.01,-0.04,-0.15,-0.04,0.01,-0.04,0.04,-0.01,-0.07,-0.01,0.01,0.08,0.01,-0.06,0.00
3,1883,-0.02,-0.08,-0.08,-0.07,-0.10,-0.02,-0.08,-0.05,-0.10,-0.05,-0.04,-0.05,-0.06,-0.06,-0.05,-0.08,-0.05,-0.06
4,1884,-0.08,-0.05,-0.09,-0.21,-0.31,-0.29,-0.21,-0.06,-0.10,-0.06,-0.10,-0.15,-0.14,-0.13,-0.06,-0.20,-0.19,-0.09


In [10]:
# Nous avons vu que certaines colonnes du tableau faisaient préalablement mention de données de type '***'
# L'opération de conversion au format float64 nous les présente désormais en tant que NaN. Il va s'agir de les remplacer.

# Quelle méthode choisir ? En effet des méthodes globales (ensemble de la colonne) ne semblent pas pertinentes car des données 
# actuelles avec des températures plus élevées pourraient polluer notre jeu de données lors de la conversion (agissant en 
# outliers). Possibilité cependant : réduire la méthode choisie à un échantillon 
# de la décennie dont font parties les données manquantes, exemple ici, nous choisirons la moyenne sur 10 ans.

# On définit des groupes de décennies
groupes_decennies = [(1880, 1890), (1890, 1900), (1900, 1910),
                     (1910, 1920), (1920, 1930), (1930, 1940),
                     (1940, 1950), (1950, 1960), (1960, 1970),
                     (1970, 1980), (1980, 1990), (1990, 2000),
                     (2000, 2010), (2010, 2020), (2020, 2024)]

# On filtre le dataframe df en fonction des groupes de décennies
df_groupes = pd.DataFrame()
for groupe in groupes_decennies:
    borne_inf, borne_sup = groupe
    df_filtre = df[(df['Year'] >= borne_inf) & (df['Year'] < borne_sup)]
    df_groupes = pd.concat([df_groupes, df_filtre]) 
    
# On sélectionne les colonnes comportant les valeurs '***' à remplacer
colonnes_a_remplacer = ['Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec','J-D', 'D-N', 'DJF', 'MAM', 'JJA', 'SON']

# On calcule la moyenne par décennie
moyennes_decennie = df_groupes.groupby(df_groupes['Year'] // 10 * 10)[colonnes_a_remplacer].mean()

# On crée une fonction pour remplacer les NaN par la moyenne par décennie
def remplacer_nan_par_moyenne(row):
    year = row['Year']
    for colonne in colonnes_a_remplacer:
        moyenne_decennie = moyennes_decennie.loc[year // 10 * 10, colonne]
        if pd.isna(row[colonne]):
            row[colonne] = round(moyenne_decennie, 2)
    return row

# Appliquer la fonction personnalisée à chaque ligne du DataFrame
df = df.apply(remplacer_nan_par_moyenne, axis=1)

df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880.0,0.00,0.04,0.06,-0.01,-0.12,-0.24,-0.16,0.07,-0.04,-0.14,0.00,0.06,-0.04,-0.12,-0.09,-0.02,-0.11,-0.06
1,1881.0,-0.08,-0.06,0.10,0.10,0.09,-0.05,-0.07,-0.02,-0.03,0.00,0.00,0.10,0.01,0.00,-0.03,0.10,-0.05,-0.01
2,1882.0,0.07,0.08,0.07,-0.01,-0.04,-0.15,-0.04,0.01,-0.04,0.04,-0.01,-0.07,-0.01,0.01,0.08,0.01,-0.06,0.00
3,1883.0,-0.02,-0.08,-0.08,-0.07,-0.10,-0.02,-0.08,-0.05,-0.10,-0.05,-0.04,-0.05,-0.06,-0.06,-0.05,-0.08,-0.05,-0.06
4,1884.0,-0.08,-0.05,-0.09,-0.21,-0.31,-0.29,-0.21,-0.06,-0.10,-0.06,-0.10,-0.15,-0.14,-0.13,-0.06,-0.20,-0.19,-0.09


In [11]:
# On vérifie s'il existe encore des valeurs manquantes dans le jeu de données :
df.isna().sum()

Year    0
Jan     0
Feb     0
Mar     0
Apr     0
May     0
Jun     0
Jul     0
Aug     0
Sep     0
Oct     0
Nov     0
Dec     0
J-D     0
D-N     0
DJF     0
MAM     0
JJA     0
SON     0
dtype: int64

In [12]:
# Pour être tout à fait complet, Il conviendrait par souci de propreté de modifier le format de la colonne Year
# il s'agit en effet ici d'une variable catégorielle dont nous ne tirerons aucun avantage statistique.
# Cependant des dates antérieures à 1970 nous empêchent de réaliser cette action.
# Nous pouvons cependant repasser Year en int64 et non plus en float64 comme auparavant.

df['Year'] = df['Year'].astype(int)
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,0.00,0.04,0.06,-0.01,-0.12,-0.24,-0.16,0.07,-0.04,-0.14,0.00,0.06,-0.04,-0.12,-0.09,-0.02,-0.11,-0.06
1,1881,-0.08,-0.06,0.10,0.10,0.09,-0.05,-0.07,-0.02,-0.03,0.00,0.00,0.10,0.01,0.00,-0.03,0.10,-0.05,-0.01
2,1882,0.07,0.08,0.07,-0.01,-0.04,-0.15,-0.04,0.01,-0.04,0.04,-0.01,-0.07,-0.01,0.01,0.08,0.01,-0.06,0.00
3,1883,-0.02,-0.08,-0.08,-0.07,-0.10,-0.02,-0.08,-0.05,-0.10,-0.05,-0.04,-0.05,-0.06,-0.06,-0.05,-0.08,-0.05,-0.06
4,1884,-0.08,-0.05,-0.09,-0.21,-0.31,-0.29,-0.21,-0.06,-0.10,-0.06,-0.10,-0.15,-0.14,-0.13,-0.06,-0.20,-0.19,-0.09


In [13]:
# On peut désormais examiner la distribution statistiques de chaque colonne de df, en faisant abstraction de la colonne Year
# qui comme nous l'avons vu n'est pas concernée.

description = df.iloc[:, 1:].describe()
display(description)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
count,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000
mean,0.046552,0.046690,0.053517,0.035655,0.022897,0.003724,0.051310,0.058069,0.054897,0.051862,0.051931,0.057724,0.044207,0.042966,0.048138,0.037586,0.037793,0.052897
std,0.318901,0.319192,0.327515,0.341995,0.360306,0.362702,0.343952,0.350379,0.350530,0.328726,0.313737,0.317743,0.319865,0.320529,0.314185,0.338079,0.342334,0.323159
min,-0.630000,-0.580000,-0.570000,-0.610000,-0.590000,-0.660000,-0.450000,-0.460000,-0.490000,-0.490000,-0.520000,-0.540000,-0.470000,-0.490000,-0.570000,-0.580000,-0.510000,-0.470000
25%,-0.190000,-0.180000,-0.180000,-0.230000,-0.270000,-0.300000,-0.230000,-0.210000,-0.230000,-0.220000,-0.180000,-0.160000,-0.210000,-0.210000,-0.180000,-0.220000,-0.240000,-0.200000
50%,-0.030000,-0.030000,-0.040000,-0.030000,-0.100000,-0.060000,-0.050000,-0.060000,-0.040000,-0.030000,-0.030000,-0.040000,-0.050000,-0.060000,-0.030000,-0.060000,-0.060000,-0.030000
75%,0.290000,0.260000,0.330000,0.320000,0.260000,0.310000,0.290000,0.330000,0.300000,0.290000,0.280000,0.320000,0.280000,0.280000,0.300000,0.300000,0.320000,0.300000
max,0.810000,0.930000,0.800000,0.970000,0.900000,0.980000,0.950000,0.920000,1.290000,0.900000,0.870000,0.850000,0.850000,0.820000,0.860000,0.840000,0.920000,0.990000


In [20]:
df['SON'].describe()

count    145.000000
mean       0.052897
std        0.323159
min       -0.470000
25%       -0.200000
50%       -0.030000
75%        0.300000
max        0.990000
Name: SON, dtype: float64